### Load Packages and Initialize Parameters

In [361]:
from twython import Twython
from datetime import datetime
import pandas as pd
import numpy as np
import re

CONSUMER_KEY = 'EH7arVyHGMo5beYuTkiUn3NUi'
CONSUMER_SECRET = 'yyjmdnexkfGr4L1assPcc7wlRYHAyqcpInIJA6ZgzbhPJljII0'
ACCESS_TOKEN = '777951321953406976-pbkpTeQ6wfNi7ChSqBwDIqnDRmfDChd'
ACCESS_SECRET = 'hqhUvRnfdsmb3kHQRiBP3kG24XqnFUDp5tTVHEfSYYSlz'

senators = ['SenShelby', 'SenatorStrange', 'lisamurkowski', 'SenDanSullivan', 'SenJohnMcCain', 'JeffFlake',
            'JohnBoozman', 'SenTomCotton', 'SenFeinstein', 'KamalaHarris', 'SenBennetCO', 'SenCoryGardner',
            'SenBlumenthal', 'ChrisMurphyCT', 'SenatorCarper', 'ChrisCoons', 'SenBillNelson', 'marcorubio',
            'SenatorIsakson', 'sendavidperdue', 'brianschatz', 'maziehirono', 'MikeCrapo', 'SenatorRisch',
            'SenatorDurbin', 'SenDuckworth', 'SenDonnelly', 'SenToddYoung', 'ChuckGrassley', 'SenJoniErnst',
            'SenPatRoberts', 'JerryMoran', 'SenateMajLdr', 'RandPaul', 'BillCassidy', 'SenJohnKennedy',
            'SenatorCollins', 'SenAngusKing', 'SenatorCardin', 'ChrisVanHollen', 'SenWarren', 'senmarkey',
            'SenStabenow', 'SenGaryPeters', 'amyklobuchar', 'SenFranken', 'SenThadCochran', 'SenatorWicker',
            'clairecmc', 'RoyBlunt', 'SenatorTester', 'SteveDaines', 'SenatorFischer', 'SenSasse', 'SenDeanHeller',
            'SenCortezMasto', 'SenatorShaheen', 'SenatorHassan', 'SenatorMenendez', 'CoryBooker', 'SenatorTomUdall',
            'MartinHeinrich', 'SenSchumer', 'SenGillibrand', 'SenatorBurr', 'SenThomTillis', 'SenJohnHoeven',
            'SenatorHeitkamp', 'SenSherrodBrown', 'SenRobPortman', 'JimInhofe', 'SenatorLankford', 'RonWyden',
            'SenJeffMerkley', 'SenBobCasey', 'SenToomey', 'SenJackReed', 'SenWhitehouse', 'LindseyGrahamSC',
            'SenatorTimScott', 'SenJohnThune', 'SenatorRounds', 'SenAlexander', 'SenBobCorker', 'JohnCornyn',
            'SenTedCruz', 'SenOrrinHatch', 'SenMikeLee', 'SenatorLeahy', 'SenSanders', 'MarkWarner', 'timkaine',
            'PattyMurray', 'SenatorCantwell', 'Sen_JoeManchin', 'SenCapito', 'SenRonJohnson', 'SenatorBaldwin',
            'SenatorEnzi', 'SenJohnBarrasso']

parties = ['Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican',
         'Republican', 'Democratic', 'Democratic', 'Democratic', 'Republican', 'Democratic', 'Democratic', 
         'Democratic', 'Democratic', 'Democratic', 'Republican', 'Republican', 'Republican', 'Democratic',
         'Democratic', 'Republican', 'Republican', 'Democratic', 'Democratic', 'Democratic', 'Republican',
         'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican',
         'Republican', 'Republican', 'Independent', 'Democratic', 'Democratic', 'Democratic', 'Democratic',
         'Democratic', 'Democratic', 'Democratic', 'Democratic', 'Republican', 'Republican', 'Democratic',
         'Republican', 'Democratic', 'Republican', 'Republican', 'Republican', 'Republican', 'Democratic',
         'Democratic', 'Democratic', 'Democratic', 'Democratic', 'Democratic', 'Democratic', 'Democratic', 
         'Democratic', 'Republican', 'Republican', 'Republican', 'Democratic', 'Democratic', 'Republican',
         'Republican', 'Republican', 'Democratic', 'Democratic', 'Democratic', 'Republican', 'Democratic',
         'Democratic', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican', 'Republican',
         'Republican','Republican', 'Republican', 'Republican', 'Democratic', 'Democratic', 'Democratic',
         'Democratic', 'Democratic', 'Democratic', 'Democratic', 'Republican', 'Republican', 'Democratic',
         'Republican', 'Republican']

### function to get tweets for a user since election day 2016

In [366]:
def get_tweets(twitter_handle):
    my_tweets = pd.DataFrame(columns=('handle','retweeted', 'favorite_count', 'retweet_count','full_text','date','time'))
    twitter = Twython(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_SECRET)

    lis = [999999999999999999] # tweet id of most recent tweet (initialized as high number)
    done = False # True when all tweets have been gathered
    for i in range(0, 16): # loop through user tweets (3200 is max you can get with API, 200 at a time)
        user_timeline = twitter.get_user_timeline(screen_name=twitter_handle, count=200, tweet_mode='extended', max_id=lis[-1])
        for tweet in user_timeline:
            #get date and time of tweet
            parsedDate = datetime.strptime(tweet['created_at'], '%a %b %d %X %z %Y').date()
            parsedTime = datetime.strptime(tweet['created_at'], '%a %b %d %X %z %Y').time()
            if parsedDate < datetime.strptime('2016 09 11', '%Y %d %m').date():
                done = True
                break
            if parsedDate > datetime.strptime('2017 23 03', '%Y %d %m').date(): 
                continue
            #fix ambersand and new lines in tweet text
            a_tweet = tweet['full_text']
            a_tweet = re.sub('&amp;','&',a_tweet)
            a_tweet = re.sub(r'\r','',a_tweet)
            #add tweet to dataframe 
            my_tweets.loc[len(my_tweets)] = [twitter_handle,'retweeted_status' in tweet,tweet['favorite_count'],tweet['retweet_count'],a_tweet,parsedDate,parsedTime]
            lis.append(tweet['id']) ## append tweet id's
        if done==True:
            break
    return my_tweets

### gather tweets for all 100 senators

In [367]:
all_tweets = pd.DataFrame(columns=('handle', 'retweeted', 'favorite_count', 'retweet_count','full_text','date','time','Party'))

print('starting scraping')
s = 0
for sen in senators:
    my_t = get_tweets(sen) #get tweets
    my_t['Party'] = np.repeat(parties[s],len(my_t)) #add party
    all_tweets = all_tweets.append(my_t) #add to master dataframe
    s += 1
    if s % 10 == 0: print(str(s) + ' out of 100 senators done')
print('done scraping')


starting scraping
10 out of 100 senators done
20 out of 100 senators done
30 out of 100 senators done
40 out of 100 senators done
50 out of 100 senators done
60 out of 100 senators done
70 out of 100 senators done
80 out of 100 senators done
90 out of 100 senators done
100 out of 100 senators done
done scraping


### write results to csv file

In [369]:
all_tweets.to_csv('all_tweets.csv',sep='`',encoding='utf-8')